In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold

In [2]:

def encode_selexSeq_data(fileprefix, 
                         data_dir="/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS", 
                features=("1mer",), w=0):
    try:
        # Load 1mer data
        ohe_file = os.path.join(data_dir, "DROSOPHILA_SELEXSEQ", "1_mer",
                                f"{fileprefix}_1mer.tsv")
        
        ohe_df = pd.read_csv(ohe_file, header=None, sep="\t")
        
        ohe_data = {row[0]: {'aff': float(row[0].split('__')[1]), 
                             'feat': row.iloc[1:].tolist()} for _, row in ohe_df.iterrows()}
        
        print(f"1mer data loaded from {ohe_file}... \n")
    
    except FileNotFoundError as fe:
        raise FileNotFoundError(f"File {ohe_file} not found: {fe}")

    combined_data = ohe_data.copy()

    # Load additional features if specified
    if len(features) > 1:
        for feature in features[1:]:
            print(f"Will load the additional feature {feature}")
            try:
                flex_feat_file = os.path.join(data_dir, "DROSOPHILA_SELEXSEQ", "DNAFlex",
                                              f"{fileprefix}_{feature}_{w}.tsv")
                flex_df = pd.read_csv(flex_feat_file, sep="\t", header=None)
                
                for ids in ohe_data.keys():
                    flex_df_row = flex_df[flex_df[0] == ids]
                    if not flex_df_row.empty:
                        flex_df_feat = np.array(flex_df_row.iloc[0, 1:].tolist()) # first column is id

                        # Scale the flex features
                        min_value = np.min(flex_df_feat)
                        std_value = np.std(flex_df_feat)
                        
                        if std_value != 0:
                            scaled_flex_feat = (flex_df_feat - min_value) / std_value
                        else:
                            scaled_flex_feat = flex_df_feat  # If std is 0, scaling is not possible

                        # Calculate interaction terms for this feature using NumPy
                        interaction_terms = scaled_flex_feat[:-1] * scaled_flex_feat[1:]

                        combined_data[ids]['feat'].extend(scaled_flex_feat.tolist() + interaction_terms.tolist())
                        
                print(f"{feature} data loaded from {flex_feat_file}... \n")
                
            except FileNotFoundError as fe:
                raise FileNotFoundError(f"File {flex_feat_file} not found: {fe}")
            except Exception as e:
                raise Exception(f"Error in extending feature {feature} from {flex_feat_file}: {e}")

    # Define X, y
    X = np.array([combined_data[x]['feat'] for x in combined_data.keys()])
    y = np.array([combined_data[x]['aff'] for x in combined_data.keys()])
    
    
    return X, y

In [219]:
def nested_ridge_cv(X, y, fileid, pos, features, outer_cv_folds=10, inner_cv_folds=5):
    
    alphas = np.linspace(1, 4000, 5000)
    
    
    outer_cv = KFold(n_splits=outer_cv_folds, shuffle=True, random_state=42)
    outer_results = []
    for train_idx, test_idx in outer_cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        
        inner_cv = KFold(n_splits=inner_cv_folds, shuffle=True, random_state=20)
        ridge_cv_model = make_pipeline(StandardScaler(with_mean=False),
                                       RidgeCV(alphas=alphas, 
                                               cv=inner_cv, 
                                               scoring='r2'))
        ridge_cv_model.fit(X_train, y_train)
        
        best_alpha = ridge_cv_model.named_steps['ridgecv'].alpha_
        y_test_pred = ridge_cv_model.predict(X_test)
        r2_test = r2_score(y_test, y_test_pred)
        
        mse_test = mean_squared_error(y_test, y_test_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        result = {
            'fileid': fileid,
            'pos': pos,
            'features': features,
            'r2_test': r2_test,
            'mse_test': mse_test,
            'mae_test': mae_test,
            'best_alpha': best_alpha
        }
        outer_results.append(result)
    return outer_results

In [ ]:
from itertools import product


ohe = ["1mer"]
features = [('1mer', ), ('1mer', "DNaseI", 'twistDisp', 'NPP', 'stiffness', 'trxDi')]
results_df_f_combined = []

features

[('1mer',), ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi')]

In [ ]:
META_FILE = "/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/METADATA/Drosophila_SELEXSeq.txt"

import pandas as pd
FILE_IDS = pd.read_csv(META_FILE, header=None)[0].to_list()
FILE_IDS

['GSM1586782_ScrWT_Exd_16mer',
 'GSM1586783_ScrR5A_Exd_16mer',
 'GSM1586784_ScrR3A_Exd_16mer',
 'GSM1586785_ScrH-12A_Exd_16mer',
 'GSM1586786_ScrH-12AR3A_Exd_16mer',
 'GSM1586787_ScrH-12AGQ_Exd_16mer',
 'GSM1586789_AntpWT_Exd_16mer',
 'GSM1586790_AntpHQ_Exd_16mer',
 'GSM1586791_AntpHT_Exd_16mer',
 'GSM1586792_AntpQT_Exd_16mer',
 'GSM1586793_AntpHQT_Exd_16mer',
 'GSM1586794_AntpScrLinkQT_Exd_16mer',
 'GSM1586795_PbFl_Exd_16mer',
 'GSM1586798_UbxIa_Exd_16mer2',
 'GSM1586799_UbxIa2_Exd_16mer',
 'GSM1586800_Lab_Exd_16mer2',
 'GSM1586801_Dfd_Exd_16mer2',
 'GSM1586802_UbxIVa_Exd_16mer2',
 'GSM1586803_AbdA_Exd_16mer2',
 'GSM1586804_AbdB_Exd_16mer2']

In [220]:
results_selexSeq = []

In [ ]:
for record in FILE_IDS:
    
    print(f"Running for file: {record} \n")
    for fset in features:

        print(f"current feature is: {fset}...\n")
#         file_id = f"{record['family']}_{record['tf']}_{record['suffix']}"
        
        # load data
        X, y = encode_selexSeq_data(fileprefix = record, features = fset)
        
        # model
        res = nested_ridge_cv(X = X, 
                              y = y, 
                              fileid = record, 
                              pos = "all",
                              features = fset,
                              outer_cv_folds=10, 
                              inner_cv_folds=10)
        
        
        print(res)
        results_selexSeq.append(res)

Running for file: GSM1586795_PbFl_Exd_16mer 

current feature is: ('1mer',)...

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/DROSOPHILA_SELEXSEQ/1_mer/GSM1586795_PbFl_Exd_16mer_1mer.tsv... 

[{'fileid': 'GSM1586795_PbFl_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.33670323338988906, 'mse_test': 0.011277673863570014, 'mae_test': 0.09291370047403165, 'best_alpha': 37.79815963192639}, {'fileid': 'GSM1586795_PbFl_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.6103037431165915, 'mse_test': 0.011129541998527899, 'mae_test': 0.08462742646712047, 'best_alpha': 48.19763952790559}, {'fileid': 'GSM1586795_PbFl_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.615084026212364, 'mse_test': 0.013537046825294448, 'mae_test': 0.06790204008459505, 'best_alpha': 41.797959591918385}, {'fileid': 'GSM1586795_PbFl_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.7518746903239247, 'mse_test': 0.003689297204862408, '

[{'fileid': 'GSM1586798_UbxIa_Exd_16mer2', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': -0.40840398734435035, 'mse_test': 0.08155899266534596, 'mae_test': 0.1399952430198389, 'best_alpha': 5.799759951990398}, {'fileid': 'GSM1586798_UbxIa_Exd_16mer2', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.3726752248183136, 'mse_test': 0.003848972561638603, 'mae_test': 0.0497920785814385, 'best_alpha': 50.597519503900784}, {'fileid': 'GSM1586798_UbxIa_Exd_16mer2', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.8498316690571579, 'mse_test': 0.0029725063793648743, 'mae_test': 0.043268809598489555, 'best_alpha': 27.39867973594719}, {'fileid': 'GSM1586798_UbxIa_Exd_16mer2', 'pos': 'all', 'features': ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi'), 'r2_test': 0.544189175699429, 'mse_test': 0.0031068093647933746, 'mae_test': 0.04

[{'fileid': 'GSM1586789_AntpWT_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4767703072257037, 'mse_test': 0.017077891921685254, 'mae_test': 0.08561967681443546, 'best_alpha': 86.59571914382877}, {'fileid': 'GSM1586789_AntpWT_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.48075661522020474, 'mse_test': 0.008481401608810958, 'mae_test': 0.07081609116708519, 'best_alpha': 97.79515903180636}, {'fileid': 'GSM1586789_AntpWT_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4168026436839628, 'mse_test': 0.016411155056551743, 'mae_test': 0.09426723628763671, 'best_alpha': 72.19643928785757}, {'fileid': 'GSM1586789_AntpWT_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.4540602240167496, 'mse_test': 0.009052977571393344, 'mae_test': 0.07510528977227066, 'best_alpha': 76.99619923984797}, {'fileid': 'GSM1586789_AntpWT_Exd_16mer', 'pos': 'all', 'features': ('1mer',), 'r2_test': 0.31612086761056457, 'mse_test': 0.017415123513099255, 'mae_test

In [200]:
ss = []


for i in results_selexSeq:
    for k in i:
        ss.append(k)
        
        
pd.DataFrame(ss).to_csv('/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/DROSOPHILA_SELEXSEQ/L2_MLR_HoxExd_updated.tsv',
                       sep="\t", index=False)